https://github.com/coqui-ai/TTS/discussions/1859

https://github.com/coqui-ai/TTS/pull/2229

In [1]:
# !pip install TTS
# !pip install numpy==1.20
# !pip install numba==0.53
# !pip install torch==1.12.1
# !pip install librosa==0.8.0
# !pip install Unidecode phonemizer
# !apt-get install espeak

In [3]:
import json
import warnings
import logging
logging.getLogger('numba').setLevel(logging.WARNING)
warnings.filterwarnings('ignore')

In [2]:
!nvidia-smi

Wed Apr 05 21:35:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.67       Driver Version: 451.67       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   37C    P8     6W /  N/A |    134MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [ ]:
!git clone https://github.com/jaywalnut310/vits
%cd vits

In [ ]:
# не устанавливать pytorch из requirements
!pip install -r requirements.txt

In [ ]:
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
#@title Download pre-trained model
!wget -nc -P ../vits-finetune/checkpoints https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/G_0.pth
!wget -nc -P ../vits-finetune/checkpoints https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/D_0.pth

In [ ]:
!python preprocess.py --filelists /content/vits/filelists/ljs_audio_text_test_filelist.txt --out_extension "new_cleaned"

In [ ]:
batchsize = 16
training_files = "/content/vits/filelists/ljs_audio_text_test_filelist.txt.new_cleaned"
validation_files = "/content/vits/filelists/ljs_audio_text_test_filelist.txt.new_cleaned"
config = json.load(open("configs/ljs_base.json"))
config['train']['batch_size'] = batchsize
config['data']['training_files'] = training_files
config['data']['validation_files'] = validation_files
with open("configs/config.json", 'w+') as f:
    json.dump(config, f, indent=4)

In [ ]:
!python -c 'from TTS.utils.downloaders import download_ljspeech; download_ljspeech("../recipes/ljspeech/");'
!ln -s ../recipes/ljspeech/LJSpeech-1.1/wavs DUMMY1

В `train.py` есть строка `utils.load_checkpoint(utils.latest_checkpoint_path(hps.model_dir, "G_*.pth"), net_g, optim_g)`, где происходит загрузка предварительно обученных моделей. Будем передавать путь к папке с параметрами моделей через аргумент `-m`. Также для удобства заменим строку 140 на:

`for batch_idx, (x, x_lengths, spec, spec_lengths, y, y_lengths) in enumerate(tqdm(train_loader)):`

Если параметры хранятся в `../drive/MyDrive/vits-finetune/checkpoints`, тогда выполним замену в файле `utils.py` строку 152 на:  
  
  `model_dir = "../drive/MyDrive/vits-finetune"`
  `model_dir = os.path.join(model_dir, args.model)`
  
 

In [ ]:
# fine-tune
!python train.py -c configs/config.json -m checkpoints